# Tutorial 5 - Global Context (Advanced)

- Agent takes in one additional parameter: `get_global_context`
- This is a function that takes in the agent's internal parameters (self) and outputs a string to the LLM to append to the prompts of any LLM-based calls internally, e.g. `get_next_subtask`, `use_llm`, `reply_to_user`
- You have full flexibility to access anything the agent knows and configure a global prompt to the agent
- This can also be used to wrap TaskGen around with a conversational interface

## Uses
- Used mainly to provide persistent variables to an agent that is not conveniently stored in `subtasks_completed`, e.g. ingredients remaining, location in grid for robot
<br></br>
- Implementing your own specific instructions to the default planner prompt
    - Implement your own memory-based RAG / global prompt instruction if you need more than what the default prompt can achieve
<br></br>
- Avoid Multiple Similar Subtasks in `subtasks_history`
    - If you have multiple similar subtask names, then it is likely the Agent can be confused and think it has already done the subtask
    - In this case, you can disambiguate by resetting the agent and store the persistent information in `shared_variables` and provide it to the agent using `get_global_context`

In [1]:
# !pip install taskgen-ai

In [2]:
# Set up API key and do the necessary imports
import os
from taskgen import *
import random

os.environ['OPENAI_API_KEY'] = '<YOUR API KEY HERE>'

# Example 1: AI-Powered Shop Assistant (with Conversational Style Interface)
- Note: This will change once `Conversation` wrapper is coded out (in progress)
- This is how we can do a basic AI-powered chatbot that helps user purchase items
- This is an illustration of how we can use global context to store chat history and other persistent variables like money remaining
- This also provides a recap on Tutorial 4: Using memory for RAG over items in `get_related_items`

In [3]:
# gets list of items and store in memory
item_list = [{"Name": "Skateboard", "item_id" : 0, "Cost": 30},
            {"Name": "Pizza", "item_id": 1, "Cost": 10},
            {"Name": "Apple Laptop", "item_id": 2, "Cost": 5000},
            {"Name": "Foldable Laptop", "item_id": 3, "Cost": 800},
            {"Name": "Apple", "item_id": 4, "Cost": 1},
            {"Name": "Machine Learning Textbook", "item_id": 5, "Cost": 100},
            {"Name": "Bicycle", "item_id": 6, "Cost": 200},
            {"Name": "Orange Juice", "item_id": 7, "Cost": 3},
            {"Name": "Coconut", "item_id": 8, "Cost": 10},
            {"Name": "Car", "item_id": 9, "Cost": 100000}]
item_memory = Memory(item_list, top_k = 3, mapper = lambda x: x, approach = 'retrieve_by_llm')

In [4]:
# Here we code more of the logic needed for function mapping from conversation and current task
# The response style to user will be handled later with strict_json
my_agent = Agent('Shop Assistant', 
f'''An assistant to help user purchase items. 
Based on Conversation, infer what the User wants.
- 1. By default, search for relevant items according to what the User wants
- 2. Purchase item if user mentions explicitly wants an item within `Items Searched` in `Latest User Message`''',
                shared_variables = {'purchased_items': [], 
                                    'money_remaining': 1000, 
                                    'item_memory': item_memory,
                                    'items_searched': [],
                                    'conversation': []},
                default_to_llm = False,
                model = 'gpt-4-turbo')

In [5]:
def get_related_items_by_instruction(shared_variables, instruction : str):
    ''' Returns all purchasable items related to the instruction. This step can also be done (and may be better for scaling up) by external vector databases '''
    
    # get all items available from memory
    item_memory = shared_variables["item_memory"]
    
    items_selected = item_memory.retrieve(instruction)
    
    # store items searched into memory if there are no duplicates
    for item in items_selected:
        if item not in shared_variables['items_searched']:
            shared_variables['items_searched'].append(item)
    
    return items_selected

my_agent.assign_functions(Function('Returns three items (item name, item_id, cost) related to the <instruction: str>. <instruction> must be item description inferred from user input. Example instructions: ["coffee", "ice cream"]', 
                 output_format = {'List of items': 'Item and cost'},
                 external_fn = get_related_items_by_instruction))

# get the agent to use the function as it contains shared_variables
my_agent.use_function("get_related_items_by_instruction", {"instruction": "I want a car"}, stateful = False)

Calling function get_related_items_by_instruction with parameters {'instruction': 'I want a car'}
> {'List of items': [{'Name': 'Car', 'item_id': 9, 'Cost': 100000}, {'Name': 'Bicycle', 'item_id': 6, 'Cost': 200}, {'Name': 'Skateboard', 'item_id': 0, 'Cost': 30}]}



{'List of items': [{'Name': 'Car', 'item_id': 9, 'Cost': 100000},
  {'Name': 'Bicycle', 'item_id': 6, 'Cost': 200},
  {'Name': 'Skateboard', 'item_id': 0, 'Cost': 30}]}

In [6]:
# clear the searched items list
my_agent.shared_variables['items_searched'] = []

In [7]:
def buy_item(shared_variables, item_id: int):
    ''' Purchases the item by item id '''
    
    # retrieve from shared variables
    money_remaining = shared_variables["money_remaining"]
    item_memory = shared_variables["item_memory"]
    purchased_items = shared_variables["purchased_items"]
    item_list = item_memory.memory
    
    # check if item_id is valid
    if not isinstance(item_id, int) or not 0 <= item_id < len(item_list):
        return f"Unable to purchase. Item id selected is not within range of 0 to {len(item_list)-1}"
    
    item = item_list[item_id]
    item_name, cost = item["Name"], item["Cost"]
    
    # if too poor to purchase, let agent know
    if cost > money_remaining:
        return f"Unable to purchase item. Available money ({money_remaining}) is lower than the cost price ({cost})"
    
    # confirm with user before purchasing
    user_input = input(f'\n\t> AI Assistant: You are about to purchase {item_name} for {cost} dollars. Proceed? Answer "Yes" to go ahead\n\t> User: ')
    if 'yes' in user_input.lower() or 'y' in user_input.lower():
        # otherwise, purchase it
        money_remaining = money_remaining - cost
        purchased_items.append(item_name)

        # store in shared variables
        shared_variables["money_remaining"] = money_remaining
        shared_variables["purchased_items"] = purchased_items

        return f"Purchase successful. Remaining money: {money_remaining}"
    
    else:
        return f"User did not want to purchase the item, and instead replied ```{user_input}```"
    
my_agent.assign_functions(Function('Purchases item corresponding to <item_id: int>', 
                 output_format = {'Status': 'Item and cost'},
                 external_fn = buy_item))

## Global Context
- Here we add in some extra information the Agent would need to make its decision
- We add in money remaining and the list of purchased items and conversational history to the context
- Here, we use TaskGen natively to handle the user query, but we let an llm take on the persona of Sherlock Holmes after the user query is done to reply user

In [8]:
def get_global_context(agent):
    ''' Outputs additional information to the agent '''
    
    # process additional context based on shared variables 
    # (this is what is called persistent variables - variables that will be updated each step)
    global_context = f'''User Money Remaining: ```{agent.shared_variables["money_remaining"]}```
Items Searched: ```{agent.shared_variables["items_searched"]}```
Items Purchased: ```{agent.shared_variables["purchased_items"]}```
Past Conversation: ```{agent.shared_variables["conversation"]}```'''
    
    return global_context

# assign this to agent's additional context
my_agent.get_global_context = get_global_context

In [9]:
persona = 'Sherlock Holmes'
conversation = []
max_conversation_length = 10
starting_msg = f'I am {persona}, an AI Assistant to help you purchase items. What would you like me to do? Type "done" to conclude'
conversation.append(f"AI Assistant: {starting_msg}")
print('\n\t> AI Assistant: '+ starting_msg)

my_agent.reset()
while True:
    # get user input
    user_input = input('\t> User: ')
    if user_input == "done": break
    
    conversation.append(f"User: {user_input}")
    
    # store only past 10 conversations
    my_agent.shared_variables['conversation'] = conversation[-max_conversation_length:]
    # do not store the past subtasks so that we do not confuse LLM - the conversation and global context is the source of truth
    my_agent.reset()
    
    # runs the agent for one subtask only (so that llm can reply user in style of persona)
    my_agent.run(f'Latest User Message: {user_input}', num_subtasks = 1)
    
    # Chatbot interface which replies the user (put this outside the agent's functions to prevent agent from hallucinating with the llm)
    res = strict_json(f'''
Global Context: ```{get_global_context(my_agent)}```
Subtasks done so far: ```{my_agent.subtasks_completed}```
Reply to User Input based on Subtasks Completed so far to carry on the Conversation as AI Assistant.
You are the one who have completed the Subtasks, inform User what you have done at the start of the reply.

Do not hallucinate anything. Base everything you say on Global Context and Subtasks Completed.
When listing items, list price as well.
Costs are in dollars. User will have no information about item_id.
Address User as You.
Ask User to type "done" if no further requests.
It is important to reply everything in the style of {persona}.''', 
                              user_input,
                              output_format = {f"Reply to User as AI Assistant": f"Reply in style of {persona}"}
    )
    agent_reply = res[f"Reply to User as AI Assistant"]
    
    conversation.append(f"AI Assistant: {agent_reply}")
    print(f'\t> AI Assistant: {agent_reply}')


	> AI Assistant: I am Sherlock Holmes, an AI Assistant to help you purchase items. What would you like me to do? Type "done" to conclude


	> User:  hot day ain't it


Subtask identified: Search for items related to cooling down on a hot day, such as ice cream.
Calling function get_related_items_by_instruction with parameters {'instruction': 'Search for items related to cooling down on a hot day, such as ice cream.'}
> {'List of items': [{'Name': 'Orange Juice', 'item_id': 7, 'Cost': 3}, {'Name': 'Coconut', 'item_id': 8, 'Cost': 10}, {'Name': 'Pizza', 'item_id': 1, 'Cost': 10}]}

	> AI Assistant: Ah, the weather does seem rather warm today. I have diligently searched for items that may aid in cooling oneself on such a day. I have found the following items: Orange Juice for $3, Coconut for $10, and Pizza for $10. Pray, do you wish to procure any of these items, or is there aught else you desire? Please type "done" if your requests are complete.


	> User:  i'd take the second one


Subtask identified: Purchase the item "Coconut" with item_id 8.
Calling function buy_item with parameters {'item_id': 8}


...

	> User:  get me the textbook


Subtask identified: Purchase the Machine Learning Textbook using its item_id.
Calling function buy_item with parameters {'item_id': 5}



	> AI Assistant: You are about to purchase Machine Learning Textbook for 100 dollars. Proceed? Answer "Yes" to go ahead
	> User:  yes


> {'Status': 'Purchase successful. Remaining money: 890'}

	> AI Assistant: Ah, dear User, I have successfully completed the task of acquiring the Machine Learning Textbook for $100 on your behalf. Your remaining funds amount to $890. Pray, do you have any further requests, or shall we consider our current business concluded? Please type "done" if your desires are fulfilled.


	> User:  done


# Example 2: Maze Navigator
- We can use the additional context to let agent know present state in a 2D grid, and obstacle positions that the agent has seen
- Task: Given current position and end position, navigate to end position using Up, Down, Left, Right

In [15]:
# These are the utility functions
def generate_grid(size, start_pos, exit_pos, obstacles):
    '''Generates a grid with obstacles'''
    grid = [[' ' for _ in range(size)] for _ in range(size)]
    grid[start_pos[0]][start_pos[1]] = 'S'  # Start
    grid[exit_pos[0]][exit_pos[1]] = 'E'  # Exit
    
    # Mark a basic path (optional, for simplicity in ensuring a path)
    # This part could be removed or replaced with a more sophisticated path marking
    path = set()
    for i in range(min(start_pos[0], exit_pos[0]), max(start_pos[0], exit_pos[0]) + 1):
        path.add((i, start_pos[1]))
    for j in range(min(start_pos[1], exit_pos[1]), max(start_pos[1], exit_pos[1]) + 1):
        path.add((exit_pos[0], j))
    
    # Randomly add obstacles
    count = 0
    while count < obstacles:
        r, c = random.randint(0, size-1), random.randint(0, size-1)
        if (r, c) not in path and grid[r][c] != 'O' and (r, c) != start_pos and (r, c) != exit_pos:
            grid[r][c] = 'O'
            count += 1
            
    return grid

def print_grid(grid):
    '''Prints the grid'''
    for row in grid:
        print(' '.join(row))
        
def check_valid_moves(cur_pos, grid, grid_size):
    '''Checks the valid moves in the grid given the cur_pos and grid_size. Returns list of valid moves within action space of Up, Down, Left, Right, Stay'''
    row, col = cur_pos
    mapping = {'Up': (-1, 0), 'Down': (1, 0), 'Left': (0, -1), 'Right': (0, 1), 'Stay': (0, 0)}
    valid_moves = []
    for key, value in mapping.items():
        row_offset, col_offset = value
        # check if valid move
        if 0 <= row+row_offset < grid_size and 0 <= col+col_offset < grid_size and grid[row+row_offset][col+col_offset] != 'O':
            valid_moves.append(key)
    return valid_moves

def update_obstacles(cur_pos, grid, grid_size, known_obstacle_pos):
    '''Returns the updated known obstacle positions in the current grid given the cur_pos and grid_size'''
    row, col = cur_pos
    mapping = {'Up': (-1, 0), 'Down': (1, 0), 'Left': (0, -1), 'Right': (0, 1), 'Stay': (0, 0)}
    for key, value in mapping.items():
        row_offset, col_offset = value
        next_row, next_col = row+row_offset, col+col_offset
        # check if valid move
        if 0 <= next_row < grid_size and 0 <= next_col < grid_size:
            # adds in obstacle if observed
            if grid[next_row][next_col] == 'O' and (next_row, next_col) not in known_obstacle_pos:
                known_obstacle_pos.append((next_row, next_col))
            # remove obstacle that is not observed
            elif (next_row, next_col) in known_obstacle_pos:
                known_obstacle_pos.remove((next_row, next_col))
    return known_obstacle_pos

In [16]:
def move(shared_variables, action: str):
    ''' Moves the agent according to the action and updates shared_variables '''
    if action not in shared_variables["next_valid_moves"]: 
        # if next move is not valid, reflect to agent
        return f'The current action of {action} is not valid. You must choose one action from {shared_variables["next_valid_moves"]}'
    mapping = {'Up': (-1, 0), 'Down': (1, 0), 'Left': (0, -1), 'Right': (0, 1), 'Stay': (0, 0)}
    
    # Retrieve from shared variables
    row, col = shared_variables["cur_pos"]
    grid = shared_variables["grid"]
    grid_size = shared_variables["grid_size"]
    known_obstacle_pos = shared_variables["known_obstacle_pos"]
    
    # Do processing for the next action
    row_offset, col_offset = mapping[action]
    newpos = (row+row_offset, col+col_offset)
    next_valid_moves = check_valid_moves(newpos, grid, grid_size)
    known_obstacle_pos = update_obstacles(newpos, grid, grid_size, known_obstacle_pos)
    
    # shift the current agent position
    grid[row][col] = ' '
    grid[row+row_offset][col+col_offset] = 'S'
    
    # Store back into shared variables
    shared_variables["cur_pos"] = newpos
    shared_variables["next_valid_moves"] = next_valid_moves
    shared_variables["known_obstacle_pos"] = known_obstacle_pos
    shared_variables["past_grid_states"].append(newpos)
    shared_variables["grid"] = grid
    
    print_grid(grid)
    
    return f'Action successful. Agent moved from {(row, col)} to {newpos}'

In [17]:
fn_list = [
    Function(f'''Moves the agent by <action: Enum['Up', 'Down', 'Left', 'Right', 'Stay']>. 
From initial position (x, y), this is what you end up with after doing actions
{{'Up': (x-1, y), 'Down': (x+1, y), 'Left': (x, y-1), 'Right': (x, y+1), 'Stay': (x, y)}}''', 
             output_format = {"Status": "Outcome of action"}, external_fn = move)
]

In [18]:
def get_global_context(agent):
    ''' Outputs additional information to the agent '''
    
    # process additional context based on shared variables (this is what is called persistent variables - variables that will be updated each step)
    global_context = f'''Agent position in grid: {agent.shared_variables["cur_pos"]}
Exit Position: {agent.shared_variables["exit_pos"]}
Last 10 Visited Grid Positions: {agent.shared_variables["past_grid_states"][:10]}
Known Obstacle Positions: {agent.shared_variables["known_obstacle_pos"]}
Next Valid Moves: {agent.shared_variables["next_valid_moves"]}'''
    
    # you can also influence how the planner performs the plan with additional details
    global_context += '''
Try to specify the action in the Subtask
Example Assigned Task: ```Navigate to (1, 1)```
Example Subtasks taking one action at a time: 'Move Down, 'Move Right' ```
'''
    
    return global_context

# Customise your grid here
# O means obstacles, S means start, E means end, blank means nothing there
grid_size = 5  # Grid size
start_pos = (random.randint(0, grid_size//2 - 1), random.randint(0, grid_size//2 - 1))  # Starting position
exit_pos = (random.randint(grid_size//2, grid_size-1), random.randint(grid_size//2, grid_size-1))  # Exit position
num_obstacles = 5  # Number of obstacles

grid = generate_grid(grid_size, start_pos, exit_pos, num_obstacles)
valid_moves = check_valid_moves(start_pos, grid, grid_size)

# Assign your agent
my_agent = Agent('Maze Navigator', 
                 f'''You are to move to the Exit Position of the maze. Task is completed when Agent's position is at Exit Position.
At each step, you can call the action Up, Down, Left, Right or Stay.
You can only move to cells without obstacles. Only take an action from Current Valid Actions.
If your previous action is invalid, choose another action.
Top left of grid is (0, 0), bottom right is {(grid_size, grid_size)}.
Grid position is referred to by (row, col)''', 
                 shared_variables = {
                    "cur_pos": start_pos,
                    "exit_pos": exit_pos,
                    "past_grid_states": [],
                    "next_valid_moves": valid_moves,
                    "known_obstacle_pos": [],
                    "grid_size": grid_size,
                    "grid": grid}, 
                 max_subtasks = 10,
                 get_global_context = get_global_context, # this is something new to store persistent states
                 default_to_llm = False).assign_functions(fn_list)

In [19]:
print('### Starting Grid ###')
print_grid(grid)
output = my_agent.run(f'Navigate to {exit_pos}')

### Starting Grid ###
         
S       O
  O O    
        O
      E O
An exception occurred: "action" not in json string output. You must use "###{key}###" to enclose the {key}.
Current invalid json format: {'Right': "Right"}
Subtask identified: Move Right towards the Exit Position
Calling function move with parameters {'action': 'Right'}
         
  S     O
  O O    
        O
      E O
> {'Status': 'Action successful. Agent moved from (1, 0) to (1, 1)'}

Subtask identified: Move Right towards the Exit Position
Calling function move with parameters {'action': 'Right'}
         
    S   O
  O O    
        O
      E O
> {'Status': 'Action successful. Agent moved from (1, 1) to (1, 2)'}

Subtask identified: Move Right towards the Exit Position(3)
Calling function move with parameters {'action': 'Right'}
         
      S O
  O O    
        O
      E O
> {'Status': 'Action successful. Agent moved from (1, 2) to (1, 3)'}

Subtask identified: Move Right towards the Exit Position(4)
Call

In [20]:
my_agent.status()

Agent Name: Maze Navigator
Agent Description: You are to move to the Exit Position of the maze. Task is completed when Agent's position is at Exit Position.
At each step, you can call the action Up, Down, Left, Right or Stay.
You can only move to cells without obstacles. Only take an action from Current Valid Actions.
If your previous action is invalid, choose another action.
Top left of grid is (0, 0), bottom right is (5, 5).
Grid position is referred to by (row, col)
Available Functions: ['end_task', 'move']
Shared Variables: ['cur_pos', 'exit_pos', 'past_grid_states', 'next_valid_moves', 'known_obstacle_pos', 'grid_size', 'grid']
Task: Navigate to (4, 3)
Subtasks Completed:
Subtask: Move Right towards the Exit Position
{'Status': 'Action successful. Agent moved from (1, 0) to (1, 1)'}

Subtask: Move Right towards the Exit Position(2)
{'Status': 'Action successful. Agent moved from (1, 1) to (1, 2)'}

Subtask: Move Right towards the Exit Position(3)
{'Status': 'Action successful. Age

## Advanced: Avoiding Multiple Similar Subtasks in `subtasks_history`

- If you have multiple similar subtask names, then it is likely the Agent can be confused and think it has already done the subtask
- In this case, you can disambiguate by resetting the agent and store the persistent information in `shared_variables` and provide it to the agent using `get_global_context`
- Has the benefit of shifting the Start State closer to End State desired by resetting the Agent's planning cycle


In [21]:
def get_global_context(agent):
    ''' Outputs additional information to the agent '''
    
    # process additional context based on shared variables (this is what is called persistent variables - variables that will be updated each step)
    global_context = f'''Agent position in grid: {agent.shared_variables["cur_pos"]}
Exit Position: {agent.shared_variables["exit_pos"]}
Last 10 Visited Grid Positions: {agent.shared_variables["past_grid_states"][:10]}
Known Obstacle Positions: {agent.shared_variables["known_obstacle_pos"]}
Current Valid Actions: {agent.shared_variables["next_valid_moves"]}'''
    
    return global_context

# Customise your grid here
# O means obstacles, S means start, E means end, blank means nothing there
grid_size = 5  # Grid size
start_pos = (random.randint(0, grid_size//2 - 1), random.randint(0, grid_size//2 - 1))  # Starting position
exit_pos = (random.randint(grid_size//2, grid_size-1), random.randint(grid_size//2, grid_size-1))  # Exit position
num_obstacles = 5  # Number of obstacles

grid = generate_grid(grid_size, start_pos, exit_pos, num_obstacles)
valid_moves = check_valid_moves(start_pos, grid, grid_size)

# Assign your agent
my_agent = Agent('Maze Navigator', 
                 f'''You are to move to the Exit Position of the maze. Task is completed when Agent's position is at Exit Position.
At each step, you can take the action Up, Down, Left, Right or Stay.

You can only move to cells without obstacles. Only take an action from Current Valid Actions.
Top left of grid is (0, 0), bottom right is {(grid_size, grid_size)}.
Grid position is referred to by (row, col)''', 
                 shared_variables = {
                    "cur_pos": start_pos,
                    "exit_pos": exit_pos,
                    "past_grid_states": [],
                    "next_valid_moves": valid_moves,
                    "known_obstacle_pos": [],
                    "grid_size": grid_size,
                    "grid": grid}, 
                 max_subtasks = 10,
                 get_global_context = get_global_context, # this is something new to store persistent states
                 default_to_llm = False).assign_functions(fn_list)

In [22]:
print('### Starting Grid ###')
print_grid(grid)

num_moves = 0
# Keep resetting subtask's history and changing start position to current position
while num_moves < 50:
    my_agent.reset()
    my_agent.run(f"Navigate to {my_agent.shared_variables['exit_pos']}", num_subtasks = 1)
    # use rule-based task checks as agent may not get it right all the time
    if my_agent.shared_variables['cur_pos'] == my_agent.shared_variables['exit_pos']: 
        my_agent.task_completed = True
        break

### Starting Grid ###
         
S   O    
    E   O
O   O    
O        
Subtask identified: Move towards the exit position (2, 2)
Calling function move with parameters {'action': 'Right'}
         
  S O    
    E   O
O   O    
O        
> {'Status': 'Action successful. Agent moved from (1, 0) to (1, 1)'}

An exception occurred: "action" not in json string output. You must use "###{key}###" to enclose the {key}.
Current invalid json format: {'Stay': "Stay"}
Subtask identified: Navigate to the Exit Position (2, 2)
Calling function move with parameters {'action': 'Down'}
         
    O    
  S E   O
O   O    
O        
> {'Status': 'Action successful. Agent moved from (1, 1) to (2, 1)'}

An exception occurred: "action" not in json string output. You must use "###{key}###" to enclose the {key}.
Current invalid json format: {'Right': "Right"}
Subtask identified: Move to the right
Calling function move with parameters {'action': 'Right'}
         
    O    
    S   O
O   O    
O        
> 

In [23]:
my_agent.status()

Agent Name: Maze Navigator
Agent Description: You are to move to the Exit Position of the maze. Task is completed when Agent's position is at Exit Position.
At each step, you can take the action Up, Down, Left, Right or Stay.

You can only move to cells without obstacles. Only take an action from Current Valid Actions.
Top left of grid is (0, 0), bottom right is (5, 5).
Grid position is referred to by (row, col)
Available Functions: ['end_task', 'move']
Shared Variables: ['cur_pos', 'exit_pos', 'past_grid_states', 'next_valid_moves', 'known_obstacle_pos', 'grid_size', 'grid']
Task: Navigate to (2, 2)
Subtasks Completed:
Subtask: Move to the right
{'Status': 'Action successful. Agent moved from (2, 1) to (2, 2)'}

Is Task Completed: True


## Can we do better? (To be added)
- LLMs are not known for their planning abilities
- Perhaps we can use an in-built planner to decide what to do for the next moves, based on what we know of the current position, exit position, obstacle positions
- Imbue the plan as part of global_context